In [1]:
import os

import requests
import tarfile
import tensorflow as tf

In [2]:
# Inception 模型 URL（地址）
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

# 模型存放地址
# 文件件不存在，创建文件夹
inception_pretrain_model_dir = 'inception_model'
if not os.path.exists(path=inception_pretrain_model_dir):
    os.mkdir(inception_pretrain_model_dir)

# 获取文件名，及路径
filename = inception_pretrain_model_url.split('/')[-1]
filepath = os.path.join(inception_pretrain_model_dir + '/', filename) # 需要加一个 ’/‘
# print(filepath)

In [3]:
# 下载模型
if not os.path.exists(filepath):
    print('downing: ', filename)
    
    # 当下载文件过大时，采用分块读取的方式
    #
    # 发送 get 响应文件只有 header，没有 body
    r = requests.get(url=inception_pretrain_model_url, stream=True) # 参数 stream=True 表示分块下载
    with open(file=filepath, mode='wb') as f:
        for chunk in r.iter_content(chunk_size=512): # 每次下载 512 个字节
            if chunk:
                f.write(chunk)
print('finsih: ', filename)

finsih:  inception-2015-12-05.tgz


In [4]:
tar = tarfile.open(name=filepath, mode='r:gz')
tar.extractall(path=inception_pretrain_model_dir)

In [5]:
# 模型结构存放文件
log_dir = 'inception_log'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
    
# classify_image_graph_def.pb 为 Goolge 训练好的模型
inception_graph_def_file = os.path.join(inception_pretrain_model_dir + '/', 'classify_image_graph_def.pb')

with tf.Session() as sess:
    # 创建一个图来来保存训练好的模型
    #
    # 将 GoogleNet 的图结构文件，以二进制编码格式打开
    with tf.gfile.FastGFile(name=inception_graph_def_file, mode='rb') as f:
        # 初始化图的连接信息
        graph_def = tf.GraphDef()
        # 得到 GoogelNet 的图信息
        graph_def.ParseFromString(f.read())
        #将 GoogelNet 的图信息导入当前图中
        tf.import_graph_def(graph_def=graph_def)
    # 保存图结构
    writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
    writer.close()